# <center>YouTube APIで動画のデータを取得する</center>

ここを参考にデータを収集してみる<br>
url:https://qiita.com/kon2/items/f09def292c3b0be2bb8e

## YouTube Data APIで再生リストから動画IDを取得

In [2]:
### サンプルコード
from googleapiclient.discovery import build
import requests
import json
from pandas import json_normalize
import pandas as pd
from datetime import date

URL = 'https://www.googleapis.com/youtube/v3/'

API_KEY = 'APIKEY'
# playlist_id = 'PL1NeGg1woXqlISJkxjgwHKgB8LmR7tk92'
# # 二郎系ラーメン
# playlist_id = 'PLRiGv_zZZiw-O-L8crBhfJrJJBNhUINel'
# # 激辛ラーメン
# playlist_id = 'PLRiGv_zZZiw_ViUw5MlfmoVFJq2NZnvAP'
# # つけ麺
# playlist_id = 'PLRiGv_zZZiw9Q7lxz5AzJmOh32No_FghX'
# # まぜそば
# playlist_id = 'PLRiGv_zZZiw-w4JSv9E3jA6Aplm-KPGfT'
# # 家系ラーメン
# playlist_id = 'PLRiGv_zZZiw94gFtfMiHyVvf1ZM_FPRUw'

playlist_id_list = ['PLRiGv_zZZiw-O-L8crBhfJrJJBNhUINel',
                               'PLRiGv_zZZiw_ViUw5MlfmoVFJq2NZnvAP',
                               'PLRiGv_zZZiw9Q7lxz5AzJmOh32No_FghX',
                               'PLRiGv_zZZiw-w4JSv9E3jA6Aplm-KPGfT',
                               'PLRiGv_zZZiw94gFtfMiHyVvf1ZM_FPRUw']

csv_list = ['jiro.csv','gekikara.csv','tsukemen.csv','mazesoba.csv','iekei.csv']

# 初期化
def initYoutube(API_KEY):
        API_SERVICE_NAME = "youtube"
        API_VERSION = "v3"
        return build(API_SERVICE_NAME, API_VERSION, developerKey=API_KEY)

def split_list(l,n):
      for idx in range(0, len(l), n):
            yield l[idx:idx + n]
    
# プレイリストから動画IDを取得する関数
def getIDlistFromPlaylist(id_,youtube):
        nextPageToken = 'start'
        response = []

        while(nextPageToken is not None):

            if(nextPageToken == 'start'):
                search_response = youtube.playlistItems().list(
                part= 'snippet',
                playlistId=id_,
                maxResults = 50,
                ).execute()
            else:
                search_response = youtube.playlistItems().list(
                part= 'snippet',
                playlistId=id_,
                maxResults = 50,
                pageToken = nextPageToken
                ).execute()

            if('nextPageToken' in search_response):
                nextPageToken = search_response['nextPageToken']
            else:
                nextPageToken = None

            for item in search_response['items']:
                response.append(item['snippet']['resourceId']['videoId'])


        response.reverse()   
        return response
    
def getCountDetails(id_,youtube):
        id_lists = split_list(id_,50)
        response = []
        
        for id_list in id_lists:
            search_response = youtube.videos().list(
            part = 'statistics,snippet',
            id=id_list,
            ).execute()
            
            response.extend(search_response['items'])
        
        return response
    
def setCountDetail(id_list,youtube,file_path):
    result = getCountDetails(id_list,youtube)
    title_list          = []
    chanel_list      = []
    id_list              = []
    link_list           = []
    published_list = []
    view_list          = []
    comment_list = []
    for item in result:
        title_list.append(item['snippet']['title'])
        chanel_list.append(item['snippet']['channelId'])
        id_list.append(item['id'])
        link_list.append('https://www.youtube.com/watch?v='+item['id'])
        published_list.append(item['snippet']['publishedAt'])
        view_list.append(int(item['statistics']['viewCount']) if 'viewCount' in item['statistics'] else -1)
        comment_list.append(int(item['statistics']['commentCount'])  if 'commentCount' in item['statistics'] else -1)
        
    df = pd.DataFrame({
        "title":title_list,
        "channel":chanel_list,
        "id":id_list,
        "link":link_list,
        "publish":published_list,
        "view":view_list,
        "comment":comment_list
    })
    
    df.to_csv(file_path,encoding="utf-8")
        
    return

youtube = initYoutube(API_KEY)

for playlist,path in zip(playlist_id_list,csv_list):
    setCountDetail(getIDlistFromPlaylist(playlist,youtube),youtube,path)

/Users/msz/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
